In [135]:
import pandas as pd
import numpy as np
import re

AllDiscussionData = pd.read_csv('data/allDiscussionsData.csv')
AllCommentsData = pd.read_csv('data/allCommentsData.csv')
UsersWithId = pd.read_csv('data/UsersWithId.csv')
last_update = pd.read_csv('data/lastUpdate_Discussions.csv')
inquisitive = 5
Curios = 20
Socrates = 40
Hero = 8
Champion = 30
Guro = 50

def read_data(new_logs_path, week):
    Newlogs = pd.read_csv(new_logs_path)
    weekNo = week
    return Newlogs, weekNo


Newlogs, weekNo = read_data('data/16.04.csv', 'week1')
Newlogs['datetime'] = pd.to_datetime(Newlogs['Time'], format="%d/%m/%y, %H:%M")


def get_deleted_object(data, object):
    deleted_objects = pd.DataFrame()
    for index, row in data.iterrows():
        if(row['Component'] == object +' deleted'):
            object_id = re.search('has deleted the '+object.lower() + ' with id (.+?) in', row['Event name'])
            if object_id:
                found = object_id.group(1)
                deleted_objects = deleted_objects.append({'object_id': found}, ignore_index=True)
    return deleted_objects


DeletedDiscussions = get_deleted_object(Newlogs, 'Discussion')
DeletedComments = get_deleted_object(Newlogs,'Post')
DeletedDiscussions.to_csv('data/DeletedPosts.csv')



In [136]:

def is_deleted_object(row, object):
    object_id = re.search('has created the ' + object + ' with id (.+?) in', row['Event name'])
    if object_id:
        found = object_id.group(1)
        if object == 'post' :
            return pd.Series(DeletedComments['object_id']).str.contains(found, regex=False).any()
        else:
            return pd.Series(DeletedDiscussions['object_id']).str.contains(found, regex=False).any()


def mark_rows(data, object):# adding 1 if the row is a Discussion
    last_update_Date =pd.to_datetime(last_update.iloc[0].TimeStamp)

    for index, row in data.iterrows():
        rowTime = row['datetime']
        if np.logical_and(row['Component'] == object +' created' , True):
        # if  (row['Component'] =='Discussion created'):
            if not is_deleted_object(row, object.lower()):
                data.loc[index, weekNo] = 1
            else :
                data.loc[index, weekNo] = 0
                print True
        else :
            data.loc[index, weekNo] = 0
    return data.reset_index()

In [137]:
UpdateData_discussions = mark_rows(Newlogs, 'Discussion')
UpdateData_comments = mark_rows(Newlogs, 'Post')

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [138]:

def Summarizer(UpdateData):
    sume = np.sum(UpdateData[weekNo])
    s2 = pd.Series(sume, name=weekNo)

    return pd.concat([s2], axis=1)

UpdateData_discussions = UpdateData_discussions.groupby('User full name').apply(lambda x: Summarizer(x)).reset_index()
UpdateData_discussions = UpdateData_discussions[UpdateData_discussions[weekNo] >0 ]
UpdateData_discussions = UpdateData_discussions.drop(['level_1'], axis=1)

UpdateData_comments = UpdateData_comments.groupby('User full name').apply(lambda x: Summarizer(x)).reset_index()
UpdateData_comments = UpdateData_comments[UpdateData_comments[weekNo] >0 ]
UpdateData_comments = UpdateData_comments.drop(['level_1'], axis=1)
# result = pd.merge(AllData, UpdateData, how='outer', on=['User full name'])


In [139]:
result_discussions = pd.merge(AllDiscussionData, UpdateData_discussions, how='outer', on=['User full name'])
result_comments = pd.merge(AllCommentsData, UpdateData_comments, how='outer', on=['User full name'])
result_comments['Sum'] = result_comments.drop(['Hero','Champion','Guro','Sum'], axis=1).sum(axis=1) #summing all weeks
result_discussions['Sum'] = result_discussions.drop(['inquisitive','Curios','Socrates','Sum'], axis=1).sum(axis=1) #summing all weeks


In [110]:
result_discussions

,Sum,inquisitive,Curios,Socrates,User full name,week1
0,2.0,NaN,NaN,NaN,Kobi Gal,2.0
1,1.0,NaN,NaN,NaN,אורי זלוטקין,1.0
2,1.0,NaN,NaN,NaN,אורי סלומון,1.0
3,2.0,NaN,NaN,NaN,איתי אסרף,2.0
4,1.0,NaN,NaN,NaN,אלון מלאך,1.0
5,1.0,NaN,NaN,NaN,אלון שניידר,1.0
6,1.0,NaN,NaN,NaN,אמיר סולומון זהבי,1.0
7,1.0,NaN,NaN,NaN,גיא אלוביץ,1.0
8,1.0,NaN,NaN,NaN,דור לוי,1.0
9,1.0,NaN,NaN,NaN,דור שרטו,1.0


In [140]:
discussions_badges = {
    "inquisitive": 5,
    "Curios": 20,
    "Socrates": 40
    }
comments_badges = {
    "Hero": 8,
    "Champion": 30,
    "Guro": 50
    }

In [141]:
def IssueBadges(result, badges_dic):
    IssueToUsers = pd.DataFrame()
    for index, row in result.iterrows():
        B1 = 0
        B2 = 0
        B3 = 0
        b1_name = list(badges_dic.keys())[0]
        b2_name = list(badges_dic.keys())[1]
        b3_name = list(badges_dic.keys())[2]

        if np.logical_and(row['Sum'] >= list(badges_dic.values())[0], row[b1_name] != 1):
            result.loc[index, b1_name] = 1
            B1=1
        if np.logical_and(row['Sum'] >= list(badges_dic.values())[1], row[b2_name] != 1):
            result.loc[index, b2_name] = 1
            B2=1
        if np.logical_and(row['Sum'] >= list(badges_dic.values())[2], row[b3_name] != 1):
            result.loc[index, b3_name] = 1
            B3=1
        IssueToUsers = IssueToUsers.append({'User full name': row['User full name'], 
                                            b1_name: B1,
                                            b2_name: B2,
                                            b3_name: B3}, ignore_index=True)
    return IssueToUsers


In [142]:
toIssueToday = IssueBadges(result_discussions, discussions_badges)


In [143]:
toIssueToday = toIssueToday.rename(columns={"User full name":'name'})

In [144]:
UsersWithId_discussions = UsersWithId.rename(columns={UsersWithId.columns[0]: "name"})
UsersWithId_discussions = pd.merge(toIssueToday, UsersWithId, how='left', on="name")


In [145]:
UsersWithId_discussions

,Curios,Socrates,name,inquisitive,id
0,0.0,0.0,Kobi Gal,0.0,NaN
1,0.0,0.0,אורי זלוטקין,0.0,302390778.0
2,0.0,0.0,אורי סלומון,0.0,316306323.0
3,0.0,0.0,איתי אסרף,0.0,322601907.0
4,0.0,0.0,אלון מלאך,0.0,207024878.0
5,0.0,0.0,אלון שניידר,0.0,312213978.0
6,0.0,0.0,אמיר סולומון זהבי,0.0,206655839.0
7,0.0,0.0,גיא אלוביץ,0.0,208726935.0
8,0.0,0.0,דור לוי,0.0,316614726.0
9,0.0,0.0,דור לוי,0.0,204459952.0
